In [1]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)


def modify_paths(config):
    # dataset.data_dir に ../ を追加
    config['dataset']['data_dir'] = os.path.join('..', config['dataset']['data_dir'])

    # dataset.split_config が存在する場合に ../ を追加
    if 'split_config' in config['dataset']:
        config['dataset']['split_config'] = os.path.join('..', config['dataset']['split_config'])

    return config

def count_parameters_in_mb(model):
    total_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
    return total_params * 4 / (1024 ** 2)  # 4バイトで換算しMBに変換


In [2]:
from omegaconf import OmegaConf
from config.modifier import dynamically_modify_train_config
from modules.fetch import fetch_model_module, fetch_data_module

In [ ]:
config_paths = [
        '../config/model/rvt_detector/rvt_frame.yaml',
        '../config/dataset/gen1/event_frame/sequence/dt_50.yaml',
        '../config/experiment/train.yaml'
    ]

configs = [OmegaConf.load(path) for path in config_paths]
merged_conf = OmegaConf.merge(*configs)
config = modify_paths(merged_conf)
dynamically_modify_train_config(config=config)


In [ ]:
from models.detection.yolox.network.backbone.darknet import CSPDarknet
from models.detection.yolox.network.backbone.pafpn import YOLOPAFPN
from models.detection.yolox.network.head.yolo_head import YOLOXHead
from models.detection.RVT.models.detection.recurrent_backbone import build_recurrent_backbone
model_ = build_recurrent_backbone(backbone_cfg=config.model.backbone)
model_.eval()
count_parameters_in_mb(model_)